In [ ]:
from io import BytesIO
import torch
from PIL import Image

import ipywidgets as widgets
import IPython.display as display
from google.colab import files

device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.hub.load("bryandlee/animegan2-pytorch:main", "generator", device=device).eval()
face2paint = torch.hub.load("bryandlee/animegan2-pytorch:main", "face2paint", device=device)

#@title Anime FaceGAN Colab app
image_format = "jpeg" #@param ["jpeg", "png"]


button = widgets.Button(description="Start")
output = widgets.Output()

uploader = widgets.FileUpload(
    accept='image/*',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

def run(b):
    button.disabled = True

    with output:
        display.clear_output()

    for fname in uploader.value:
        bytes_in = uploader.value[fname]['content']

        im_in = Image.open(BytesIO(bytes_in)).convert("RGB")
        im_out = face2paint(model, im_in, side_by_side=False)

        buffer_out = BytesIO()
        im_out.save(buffer_out, format=image_format)

        bytes_out = buffer_out.getvalue()
        wi1 = widgets.Image(value=bytes_in, format=image_format)
        wi2 = widgets.Image(value=bytes_out, format=image_format)

        wi1.layout.max_width = '500px'
        wi1.layout.max_height = '500px'
        wi2.layout.max_width = '500px'
        wi2.layout.max_height = '500px'

        ## Side by side thanks to HBox widgets
        sidebyside = widgets.HBox([wi1, wi2])
        ## Finally, show.
        with output:
            display.display(sidebyside)
    
    button.disabled = False

button.on_click(run)
display.display(uploader, button, output)